In [1]:
import torch

from garage import wrap_experiment
from garage.envs import GymEnv
from garage.experiment.deterministic import set_seed
from garage.sampler import LocalSampler
from garage.torch.algos import TRPO
from garage.torch.algos import VPG
from garage.torch.policies import GaussianMLPPolicy
from garage.torch.value_functions import GaussianMLPValueFunction
from garage.trainer import Trainer

@wrap_experiment
def trpo_pendulum(ctxt=None, seed=1):
    """Train TRPO with InvertedDoublePendulum-v2 environment.
    Args:
        ctxt (garage.experiment.ExperimentContext): The experiment
            configuration used by Trainer to create the snapshotter.
        seed (int): Used to seed the random number generator to produce
            determinism.
    """
    set_seed(seed)
    env = GymEnv('MountainCarContinuous-v0')

    trainer = Trainer(ctxt)

    policy = GaussianMLPPolicy(env.spec,
                               hidden_sizes=[32, 32],
                               hidden_nonlinearity=torch.tanh,
                               output_nonlinearity=None)

    value_function = GaussianMLPValueFunction(env_spec=env.spec,
                                              hidden_sizes=(32, 32),
                                              hidden_nonlinearity=torch.tanh,
                                              output_nonlinearity=None)

    sampler = LocalSampler(agents=policy,
                           envs=env,
                           max_episode_length=env.spec.max_episode_length)

    algo = TRPO(env_spec=env.spec,
                policy=policy,
                value_function=value_function,
                sampler=sampler,
                discount=0.99,
                center_adv=False)

    trainer.setup(algo, env)
    trainer.train(n_epochs=500, batch_size=1024)

In [2]:
trpo_pendulum(seed=1234)

2022-08-23 10:37:18 | [trpo_pendulum] Logging to d:\Github\DRSOM-for-RL\data/local/experiment/trpo_pendulum_4


d:\Anaconda\lib\site-packages\garage\experiment\deterministic.py:36: UserWarning: Enabeling deterministic mode in PyTorch can have a performance impact when using GPU.
  warnings.warn(


2022-08-23 10:37:18 | [trpo_pendulum] Obtaining samples...
flat loss grads is: 
tensor([ 1.4506e-01, -6.3618e-03, -9.6141e-05,  ..., -2.9816e-03,
         4.6496e-03, -3.1764e-02])
G is: 
tensor([[0.0319, 0.0986],
        [0.0986, 0.4065]])
eig is:
tensor([[0.0075, 0.0000],
        [0.4308, 0.0000]])
loss before is:
tensor(3.4138, grad_fn=<NegBackward>)
loss now is:
tensor(3.4034, grad_fn=<NegBackward>)
----------------------------
2022-08-23 10:37:19 | [trpo_pendulum] epoch #0 | Saving snapshot...
2022-08-23 10:37:19 | [trpo_pendulum] epoch #0 | Saved
2022-08-23 10:37:19 | [trpo_pendulum] epoch #0 | Time 1.00 s
2022-08-23 10:37:19 | [trpo_pendulum] epoch #0 | EpochTime 1.00 s
-----------------------------------  -------------
Evaluation/AverageDiscountedReturn    -10.8561
Evaluation/AverageReturn             -108.645
Evaluation/Iteration                    0
Evaluation/MaxReturn                 -103.75
Evaluation/MinReturn                 -113.54
Evaluation/NumEpisodes                